In [2]:
from Extract_ephys_from_struct import ExtractEphysData, calculate_mean_responses, ResponseDistributionPlotter
from Analysis_for_ephys import * # import all functions from Analysis_for_ephys.py

In [3]:
#from plotting_functions import plot_spike_distribution_boxplots, plot_spike_distribution, plot_psth, plot_raster

#now test the class
#initialize the class
EED = ExtractEphysData('/Users/cresp1el/Documents/MATLAB/lmc_20ms_data.mat')

In [3]:
#test the methods of the class
EED.load_matfiles_printdata()
EED.get_original_cellid('4bc09822d0832f7c00be68d2fb7a01dc') #get the original cell id for a given unit id
EED.get_pre_data('4bc09822d0832f7c00be68d2fb7a01dc') #get the pre data for a given unit id


Group name: Lmc_noopsin
  Recording name: lmc_noch_1_3096_rec1 - Total units: 5
  Recording name: lmc_noch_1_3096_rec2 - Total units: 4
Group name: Lmc_opsin
  Recording name: lmc_ch_1_3094_rec1 - Total units: 7
  Recording name: lmc_ch_1_3094_rec2 - Total units: 3
  Recording name: lmc_ch_2_3093_rec1 - Total units: 4


{'4bc09822d0832f7c00be68d2fb7a01dc': {'FRs_baseline': [array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0.]),
   array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [ ]:
for group in EED.get_group_names():
    for recoring in EED.get_recording_names(group):
        print(recoring)
        

In [8]:
EED.construct_stimulus_table('lmc_ch_1_3094_rec1')

KeyError: 'Stim_Onset_samples'

In [ ]:
unit_data = EED.get_unit_level_data()

In [ ]:
key_names = EED.get_unit_data_keys('4bc09822d0832f7c00be68d2fb7a01dc')
print(key_names)

In [ ]:
unit_data_df = EED.get_unit_table()
unit_data_df.head()# Print the first few rows of the DataFrame to check the output
#check the shape of the dataframe
unit_data_df.shape

In [ ]:
# Create an instance of the UnitDataAnalysis class with the unit data DataFrame
analysis = UnitDataAnalysis(unit_data_df)
analysis.convert_samples2seconds()


In [ ]:
#access thhe data in the analysis class 
analysis.unit_data['Group'].unique() #get the unique values in the group column

In [ ]:
analysis.calculate_and_plot_firing_rate_matrix(bin_size=10)
    
    

In [ ]:
test = calculate_mean_responses(EED)

In [ ]:
print(test.keys()) #these are the group names
print(test['Lmc_opsin'].keys()) #these are the pre and post epoch names for each group
print(test['Lmc_opsin']['Pre']) #these are the cell names for each epoch

In [ ]:
# Assuming `mean_responses_data` is the dictionary obtained from the calculate_mean_responses function
plotter = ResponseDistributionPlotter(test)
plotter.plot_distribution('Lmc_opsin', 'Pre', 'Pooled', bins=100, overlay=True, phase='early') 
plotter.plot_distribution('Lmc_opsin', 'Post', 'Pooled', bins=100, overlay=True, phase='early')

# Assuming `mean_responses_data` is the dictionary obtained from the calculate_mean_responses function
plotter = ResponseDistributionPlotter(test)
plotter.plot_distribution('Lmc_opsin', 'Pre', 'Pooled', bins=100, overlay=True, phase='late') 
plotter.plot_distribution('Lmc_opsin', 'Post', 'Pooled', bins=100, overlay=True, phase='late')

In [ ]:
# plotter.plot_distribution('Lmc_opsin', epoch=None, stim_level='Zero', overlay=True)
#plot the distribution for pre and post stim levels separately
plotter.plot_distribution('Lmc_opsin', epoch='Pre', stim_level='Zero', overlay=True)
plotter.plot_distribution('Lmc_opsin', epoch='Post', stim_level='Zero', overlay=True)

# Plotting 'Pre' and 'Post' distributions on top of each other by setting overlay=True 
#loop over the 'Zero', 'Low', 'Mid', 'Max', or 'Pooled' stim levels to plot the distributions for each stim level
for stim_level in ['Zero', 'Low', 'Mid', 'Max', 'Pooled']: 
    plotter.plot_distribution('Lmc_opsin', epoch=None, stim_level=stim_level, overlay=True)
    
    

In [ ]:
#plot the pre and post for the low separated 
plotter.plot_distribution('Lmc_opsin', epoch='Pre', stim_level='Low', bins=100)
plotter.plot_distribution('Lmc_opsin', epoch='Post', stim_level='Low', bins=100)

#plot the pre and post for the high separated
plotter.plot_distribution('Lmc_opsin', epoch='Pre', stim_level='Max', bins=100)
plotter.plot_distribution('Lmc_opsin', epoch='Post', stim_level='Max', bins=100)


In [ ]:
import matplotlib.pyplot as plt
from collections import defaultdict

def plot_box_and_whisker(data, group_name, stim_levels=['Zero', 'Low', 'Mid', 'Max', 'Pooled'], epoch='both', overlay=False):
    """
    Plots a box and whisker plot of the mean responses for the specified group and stimulation levels.

    Parameters:
    - data (dict): The data dictionary containing the mean responses.
    - group_name (str): The name of the group to plot data for.
    - stim_levels (list or str): The stimulation levels to plot data for (e.g., ['Zero', 'Low', 'Mid', 'Max', 'Pooled'] or 'Zero'). Default is all levels.
    - epoch (str): The epoch to plot data for ('Pre', 'Post', or 'both'). Default is 'both'.
    - overlay (bool): Whether to overlay the 'Pre' and 'Post' data on a single plot. Default is False.
    """

    if isinstance(stim_levels, str):
        stim_levels = [stim_levels]

    # Get the mean responses for the specified group
    group_data = data[group_name]

    # Get the data to plot
    data_to_plot = defaultdict(list)
    epochs_to_plot = ['Pre', 'Post'] if epoch == 'both' else [epoch]
    for epoch in epochs_to_plot:
        for stim_level in stim_levels:
            stim_data = [unit_data[f'{epoch}_{stim_level}'] for unit_data in group_data[epoch]]
            data_to_plot[epoch].append(stim_data)

    # Create a new figure
    plt.figure()

    # Create a box plot for each epoch
    colors = {'Pre': 'grey', 'Post': 'blue'}
    for i, (epoch, stim_data) in enumerate(data_to_plot.items()):
        position = range(i * (len(stim_levels) + 1) + 1, (i + 1) * (len(stim_levels) + 1))
        bp = plt.boxplot(stim_data, positions=position, labels=stim_levels if i == 0 else [''] * len(stim_levels), patch_artist=True)
        
        # Set colors
        for box in bp['boxes']:
            box.set_facecolor(colors[epoch])
        
    # Set the plot title and labels
    plt.title(f'{group_name} Mean Responses')
    plt.ylabel('Mean Response')
    plt.xlabel('Stimulation Level')
    
    # Adding epoch labels
    if epoch == 'both':
        mid_point = len(stim_levels) / 2
        plt.text(mid_point, plt.gca().get_ylim()[1], 'Pre', ha='center', va='center', fontsize=12, fontweight='bold')
        plt.text(len(stim_levels) + 1 + mid_point, plt.gca().get_ylim()[1], 'Post', ha='center', va='center', fontsize=12, fontweight='bold')

    # Show the plot
    if not overlay:
        plt.show()

# Usage example:
# Assuming `data_dict` is your data dictionary obtained from `calculate_mean_responses` function
plot_box_and_whisker(test, 'Lmc_opsin', 'Zero', 'both')





In [ ]:
import matplotlib.pyplot as plt
from collections import defaultdict

def plot_box_and_whisker(data, group_name, stim_levels=['Zero', 'Low', 'Mid', 'Max', 'Pooled'], epoch='both', overlay=False):
    """
    Plots a box and whisker plot of the mean responses for the specified group and stimulation levels.

    Parameters:
    - data (dict): The data dictionary containing the mean responses.
    - group_name (str): The name of the group to plot data for.
    - stim_levels (list or str): The stimulation levels to plot data for (e.g., ['Zero', 'Low', 'Mid', 'Max', 'Pooled'] or 'Zero'). Default is all levels.
    - epoch (str): The epoch to plot data for ('Pre', 'Post', or 'both'). Default is 'both'.
    - overlay (bool): Whether to overlay the 'Pre' and 'Post' data on a single plot. Default is False.
    """

    if isinstance(stim_levels, str):
        stim_levels = [stim_levels]

    # Get the mean responses for the specified group
    group_data = data[group_name]

    # Get the data to plot
    data_to_plot = defaultdict(list)
    epochs_to_plot = ['Pre', 'Post'] if epoch == 'both' else [epoch]
    for epoch in epochs_to_plot:
        for stim_level in stim_levels:
            stim_data = [unit_data[f'{epoch}_{stim_level}'] for unit_data in group_data[epoch]]
            data_to_plot[epoch].append(stim_data)

    # Create a new figure
    plt.figure()

    # Create a box plot for each epoch
    colors = {'Pre': 'grey', 'Post': 'blue'}
    for i, (epoch, stim_data) in enumerate(data_to_plot.items()):
        position = range(i * (len(stim_levels) + 1) + 1, (i + 1) * (len(stim_levels) + 1))
        labels = [f'{epoch}{level}' for level in stim_levels]
        bp = plt.boxplot(stim_data, positions=position, labels=labels, patch_artist=True)
        
        # Set colors
        for box in bp['boxes']:
            box.set_facecolor(colors[epoch])
        
    # Set the plot title and labels
    plt.title(f'{group_name} Mean Responses')
    plt.ylabel('Mean Response')
    plt.xlabel('Stimulation Level Epoch')
    
    # Show the plot
    if not overlay:
        plt.show()



# Usage example:

# Assuming `data_dict` is your data dictionary obtained from `calculate_mean_responses` function
plot_box_and_whisker(test, 'Lmc_opsin', 'Max', 'both')


In [ ]:
import matplotlib.pyplot as plt
from collections import defaultdict

def plot_box_and_whisker(data, group_name, stim_levels=['Zero', 'Low', 'Mid', 'Max', 'Pooled'], epoch='both', overlay=False, plot_hist=False):
    """
    Plots a box and whisker plot of the mean responses for the specified group and stimulation levels.

    Parameters:
    - data (dict): The data dictionary containing the mean responses.
    - group_name (str): The name of the group to plot data for.
    - stim_levels (list or str): The stimulation levels to plot data for (e.g., ['Zero', 'Low', 'Mid', 'Max', 'Pooled'] or 'Zero'). Default is all levels.
    - epoch (str): The epoch to plot data for ('Pre', 'Post', or 'both'). Default is 'both'.
    - overlay (bool): Whether to overlay the 'Pre' and 'Post' data on a single plot. Default is False.
    - plot_hist (bool): Whether to plot the histogram distribution. Default is False.
    """

    if isinstance(stim_levels, str):
        stim_levels = [stim_levels]

    # Get the mean responses for the specified group
    group_data = data[group_name]

    # Get the data to plot
    data_to_plot = defaultdict(list)
    epochs_to_plot = ['Pre', 'Post'] if epoch == 'both' else [epoch]
    for epoch in epochs_to_plot:
        for stim_level in stim_levels:
            stim_data = [unit_data[f'{epoch}_{stim_level}'] for unit_data in group_data[epoch]]
            data_to_plot[epoch].append(stim_data)

    # Create a new figure
    plt.figure()

    # Create a box plot for each epoch
    colors = {'Pre': 'grey', 'Post': 'blue'}
    for i, (epoch, stim_data) in enumerate(data_to_plot.items()):
        position = range(i * (len(stim_levels) + 1) + 1, (i + 1) * (len(stim_levels) + 1))
        labels = [f'{epoch}_{level}' for level in stim_levels]
        bp = plt.boxplot(stim_data, positions=position, labels=labels, patch_artist=True)
        
        # Set colors
        for box in bp['boxes']:
            box.set_facecolor(colors[epoch])
        
    # Set the plot title and labels
    plt.title(f'{group_name} Mean Responses')
    plt.ylabel('Mean Response')
    plt.xlabel('Stimulation Level Epoch')
    
    # Rotate x-axis labels for better visibility
    plt.xticks(rotation=45)
    
    # Plot the histogram distribution if specified
    if plot_hist:
        plt.figure()
        for epoch in epochs_to_plot:
            for stim_level in stim_levels:
                stim_data = [unit_data[f'{epoch}_{stim_level}'] for unit_data in group_data[epoch]]
                plt.hist(stim_data, alpha=0.5, label=f'{epoch}_{stim_level}', color=colors[epoch], histtype='step')
        plt.xlabel('Mean Response')
        plt.ylabel('Frequency')
        plt.title(f'{group_name} Response Distributions')
        plt.legend()
    
    # Show the plot
    if not overlay:
        plt.show()


# Usage example:
# Assuming `data_dict` is your data dictionary obtained from `calculate_mean_responses` function
plot_box_and_whisker(test, 'Lmc_opsin', 'Zero', 'both', plot_hist=True)

# Assuming `data_dict` is your data dictionary obtained from your data processing
plot_box_and_whisker(test, group_name='Lmc_opsin')


In [ ]:
# Example 1: Basic Usage with Default Parameters
plot_box_and_whisker(test, group_name='Lmc_opsin')

#Example 2: Plotting for a Single Epoch with Histograms
plot_box_and_whisker(test, group_name='Lmc_opsin', epoch='Pre', plot_hist=True)

#Example 3: Plotting for Specific Stimulation Levels 
plot_box_and_whisker(test, group_name='Lmc_opsin', stim_levels=['Zero', 'Max'], plot_hist=True)

#Example 4: Overlaying 'Pre' and 'Post' Data on a Single Plot
plot_box_and_whisker(test, group_name='Lmc_opsin', overlay=True)

# Example 5: Plotting for a Single Stimulation Level
plot_box_and_whisker(test, group_name='Lmc_opsin', stim_levels='Zero', plot_hist=True)

In [ ]:
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt

def plot_regression(data, stimulation='Pooled', overlay=True, log_scale=False):
    
    pre_early = []
    pre_late = []
    post_early = []
    post_late = []

    for group in data.values():
        for epoch_data in group.values():
            for record in epoch_data:
                pre_early.append(record[f'Pre_{stimulation}_early'])
                pre_late.append(record[f'Pre_{stimulation}_late'])
                post_early.append(record[f'Post_{stimulation}_early'])
                post_late.append(record[f'Post_{stimulation}_late'])

    fig, ax = plt.subplots(1, 2 if not overlay else 1, figsize=(10, 5))

    if log_scale:
        pre_early = [np.log(x+1) for x in pre_early]
        pre_late = [np.log(x+1) for x in pre_late]
        post_early = [np.log(x+1) for x in post_early]
        post_late = [np.log(x+1) for x in post_late]

    if overlay:
        ax.scatter(pre_early, pre_late, color='grey', alpha=0.5, label='Pre')
        ax.scatter(post_early, post_late, color='blue', alpha=0.5, label='Post')
        
        # Adding simple linear regression lines
        slope, intercept, _, _, _ = stats.linregress(pre_early, pre_late)
        ax.plot(pre_early, np.array(pre_early)*slope + intercept, color='grey')
        
        slope, intercept, _, _, _ = stats.linregress(post_early, post_late)
        ax.plot(post_early, np.array(post_early)*slope + intercept, color='blue')
        
        ax.set_xlabel('Early Phase')
        ax.set_ylabel('Late Phase')
    else:
        ax[0].scatter(pre_early, pre_late, color='grey', alpha=0.5, label='Pre')
        ax[1].scatter(post_early, post_late, color='blue', alpha=0.5, label='Post')
        
        # Adding simple linear regression lines
        slope, intercept, _, _, _ = stats.linregress(pre_early, pre_late)
        ax[0].plot(pre_early, np.array(pre_early)*slope + intercept, color='grey')
        
        slope, intercept, _, _, _ = stats.linregress(post_early, post_late)
        ax[1].plot(post_early, np.array(post_early)*slope + intercept, color='blue')
        
        ax[0].set_xlabel('Early Phase')
        ax[0].set_ylabel('Late Phase')
        ax[1].set_xlabel('Early Phase')
        ax[1].set_ylabel('Late Phase')
        
        ax[0].legend()

    plt.legend()
    plt.show()

# Example commands to test the function
# Replace 'test' with your actual data dictionary
plot_regression(test, stimulation='Pooled', overlay=True, log_scale=True)
plot_regression(test, stimulation='Pooled', overlay=False, log_scale=True)
plot_regression(test, stimulation='Low', overlay=True, log_scale=False)


In [ ]:
print(type(test['Lmc_opsin']['Pre'])) #prints the type of the data
print(len(test['Lmc_opsin']['Pre'])) #print the number of units in the pre epoch of the Lmc_opsin group
print(test['Lmc_opsin']['Pre'][0].keys()) #print the keys of the first element in the list 

In [ ]:
print(type(test['Lmc_opsin']['Pre'])) #prints the type of the data
print(len(test['Lmc_opsin']['Pre'])) #print the number of units in the pre epoch of the Lmc_opsin group
print(test['Lmc_opsin']['Pre'][0].keys()) #print the keys of the first element in the list 

In [ ]:
from scipy import stats
import numpy as np

def calculate_pearson_correlations(EED, group_name=None):
    """
    Calculate the Pearson correlation coefficients using the individual trial responses from both the early and late windows during both the pre and post epochs for each unit.

    Parameters:
    EED (object): The object containing the electrophysiology data.
    group_name (str, optional): The name of the group to analyze. Defaults to None, in which case all groups are analyzed.

    Returns:
    dict: A dictionary containing the Pearson correlation coefficients for each unit.
    """
    
    # Get list of group names
    group_names = [group_name] if group_name else EED.group_names
    
    # Dictionary to store the Pearson correlation coefficients for each group
    pearson_correlations = {}
    
    # Loop through each group
    for group in group_names:
        recording_names = EED.get_recording_names(group)
        
        # Dictionary to store the Pearson correlation coefficients for all units in the current group
        group_pearson_correlations = {'Pre': [], 'Post': []}
        
        # Loop through each recording
        for recording in recording_names:
            cellid_names = EED.get_cellid_names(group, recording)
            
            # Loop through each cell ID
            for cell_id in cellid_names:
                # Get the pre and post stim data
                data = EED.get_pre_post_data(group, recording, cell_id)
                
                # Define stimulation levels and pooled stimulation levels
                stim_levels = ['Zero', 'Low', 'Mid', 'Max', 'Pooled']
                
                # Dictionary to store the Pearson correlation coefficients for the current unit
                unit_pearson_correlations = {'Recording': recording, 'CellID': cell_id}
                
                # Loop through each epoch (pre and post) and stimulation level to calculate Pearson correlation coefficients
                for epoch in ['Pre', 'Post']: #loop through the pre and post epochs
                    for stim_level in stim_levels: #loop through the stim levels
                        if stim_level == 'Pooled': #if the stim level is pooled, then use all stim levels
                            stim_indices = [1, 2, 3] #indices for the stim levels
                        else: #otherwise, use the stim level specified
                            stim_indices = [stim_levels.index(stim_level)] #create a list with the index of the stim level
                        
                        # Get spike trains for the current stimulation level
                        spiketrains = np.concatenate([data[epoch]['SpikeTrains_for_PSTHs'][i] for i in stim_indices], axis=0) #list comprehension to get the spike trains for the stim level by looping through the stim indices
                        
                        # Extract spike data for the early phase (0-50 ms post-stimulus)
                        early_phase = spiketrains[:, 500:550]  # Adjust indices as necessary
                        
                        # Extract spike data for the late phase (100-700 ms post-stimulus)
                        late_phase = spiketrains[:, 600:1200]  # Adjust indices as necessary
                        
                        # Calculate the total number of spikes in each trial during the early and late phases
                        early_response = early_phase.sum(axis=1) #sum across the rows which are the trials for the early phase
                        late_response = late_phase.sum(axis=1) #sum across the rows which are the trials for the late phase
                        
                        # Calculate the Pearson correlation coefficient for the early and late responses
                        corr, _ = stats.pearsonr(early_response, late_response)
                        
                        # Add the Pearson correlation coefficient to the dictionary
                        unit_pearson_correlations[f'{epoch}_{stim_level}_Pearson_Correlation'] = corr
                
                # Calculate the difference between post and pre Pearson correlation coefficients for each stimulation level
                for stim_level in stim_levels:
                    pre_corr = unit_pearson_correlations[f'Pre_{stim_level}_Pearson_Correlation']
                    post_corr = unit_pearson_correlations[f'Post_{stim_level}_Pearson_Correlation']
                    unit_pearson_correlations[f'{stim_level}_Correlation_Difference'] = post_corr - pre_corr
                
                # Add the Pearson correlation coefficients for the current unit to the list
                group_pearson_correlations['Pre'].append(unit_pearson_correlations)
                group_pearson_correlations['Post'].append(unit_pearson_correlations)
        
        # Add the Pearson correlation coefficients for the current group to the dictionary
        pearson_correlations[group] = group_pearson_correlations
    
    return pearson_correlations


# use the function
pearson_correlations = calculate_pearson_correlations(EED, group_name=None)

In [ ]:
pearson_correlations.keys() #prints the keys of the dictionary
pearson_correlations['Lmc_opsin'].keys() 
pearson_correlations['Lmc_opsin']['Pre'][0]

In [ ]:
print(list(pearson_correlations.values())[0].keys())

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np

def plot_correlation_differences(correlation_data, stimulation_level):
    """
    Plot the correlation differences for all units, ranked from least to most, for a specified stimulation level.

    Parameters:
    correlation_data (dict): The dictionary containing the Pearson correlation data for each unit.
    stimulation_level (str): The stimulation level to plot (one of 'Zero', 'Low', 'Mid', 'Max', 'Pooled').

    Returns:
    None: The function will plot the data but not return any values.
    """
    
    # Loop through each group in the correlation data
    for group_name, group_data in correlation_data.items():
        
        # List to store all units and their correlation differences
        all_units_correlation_differences = []

        # Loop through each unit in the 'Pre' epoch to get its correlation difference for the specified stimulation level
        for unit in group_data['Pre']:
            # Get the correlation difference for the current unit and the specified stimulation level
            correlation_difference = unit.get(f'{stimulation_level}_Correlation_Difference') # Returns None if the key doesn't exist
            
            # Skip units with invalid correlation differences
            if correlation_difference is None or np.isnan(correlation_difference):
                continue
            
            # Get the cell ID for the current unit
            cell_id = unit['CellID']
            
            # Add the unit's cell ID and correlation difference to the list
            all_units_correlation_differences.append((cell_id, correlation_difference))
        
        # Print the list of units and their correlation differences
        print(all_units_correlation_differences)  

        # Sort the units by their correlation differences from least to most
        sorted_units = sorted(all_units_correlation_differences, key=lambda x: x[1])
        
        # Get the cell IDs and correlation differences as separate lists
        cell_ids, correlation_differences = zip(*sorted_units)
        
        # Create a bar plot of the correlation differences
        plt.figure(figsize=(10, 6))
        
        # Create a colormap for the bar colors
        norm = mcolors.Normalize(vmin=-1, vmax=1)
        cmap = plt.cm.get_cmap('coolwarm')
        
        # Create a bar plot with colors based on the correlation differences
        plt.bar(range(len(correlation_differences)), correlation_differences, tick_label=cell_ids, color=cmap(norm(correlation_differences)))
        
        # Add a colorbar to the plot
        sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
        sm.set_array([])
        plt.colorbar(sm, label='Correlation Difference')
        
        # Add labels and title to the plot
        plt.xlabel('Cell ID')
        plt.ylabel('Correlation Difference')
        plt.title(f'Correlation Differences for {stimulation_level} Stimulation in {group_name} group')
        
        # Rotate the x-axis labels for better readability
        plt.xticks(rotation=90)
        
        # Display the plot
        plt.show()



plot_correlation_differences(pearson_correlations, 'Mid')

#loop over the stimulation levels
for stimulation_level in ['Zero', 'Low', 'Mid', 'Max', 'Pooled']:
    plot_correlation_differences(pearson_correlations, stimulation_level)


In [ ]:
from scipy import stats
import numpy as np

def calculate_global_pearson_correlations(EED, group_name=None):
    """
    Calculate the Pearson correlation coefficients using the mean responses from both the early and late windows during both the pre and post epochs for each unit.

    Parameters:
    EED (object): The object containing the electrophysiology data.
    group_name (str, optional): The name of the group to analyze. Defaults to None, in which case all groups are analyzed.

    Returns:
    dict: A dictionary containing the Pearson correlation coefficients for each unit.
    """
    
    group_names = [group_name] if group_name else EED.group_names
    
    detailed_data = {}
    
    for group in group_names:
        recording_names = EED.get_recording_names(group)
        
        for recording in recording_names:
            cellid_names = EED.get_cellid_names(group, recording)
            
            for cell_id in cellid_names:
                data = EED.get_pre_post_data(group, recording, cell_id)
                
                stim_levels = ['Zero', 'Low', 'Mid', 'Max', 'Pooled']
                
                for epoch in ['Pre', 'Post']:
                    for stim_level in stim_levels:
                        if stim_level == 'Pooled':
                            stim_indices = [1, 2, 3]
                        else:
                            stim_indices = [stim_levels.index(stim_level)]
                        
                        spiketrains = np.concatenate([data[epoch]['SpikeTrains_for_PSTHs'][i] for i in stim_indices], axis=0)
                        
                        early_phase = spiketrains[:, 500:550]
                        late_phase = spiketrains[:, 600:1200]
                        
                        early_response = early_phase.sum(axis=1) / early_phase.shape[1]
                        late_response = late_phase.sum(axis=1) / late_phase.shape[1]
                        
                        early_mean = early_response.mean()
                        late_mean = late_response.mean()
                        
                        detailed_data.setdefault(group, {}).setdefault(recording, {}).setdefault(cell_id, {}).setdefault(epoch, {}).setdefault(stim_level, {
                            'early_phase': early_phase,
                            'late_phase': late_phase,
                            'early_response': early_response,
                            'late_response': late_response,
                            'early_mean': early_mean,
                            'late_mean': late_mean
                        })
    
    return detailed_data

# use the function
# detailed_data = calculate_global_pearson_correlations(EED, group_name=None)


# use the function
pearson_correlations_global = calculate_global_pearson_correlations(EED, group_name=None)


In [ ]:
pearson_correlations_global.keys() #prints the keys of the dictionary

In [ ]:
from scipy import stats

def calculate_pearson_correlations(EED, detailed_data):
    """
    Calculate the Pearson correlation coefficients using the mean responses from the detailed data.

    Parameters:
    EED (object): The object containing the electrophysiology data.
    detailed_data (dict): The dictionary containing the detailed data including early and late mean responses.

    Returns:
    dict: A dictionary containing the Pearson correlation coefficients for each stimulation level and epoch.
    dict: A dictionary containing the cell IDs that were removed due to NaN values.
    """

    pearson_correlations = {}
    removed_cell_ids = {}

    # Get list of group names
    group_names = EED.group_names

    # Define stimulation levels
    stim_levels = ['Zero', 'Low', 'Mid', 'Max', 'Pooled']

    # Loop through each group
    for group in group_names:
        group_correlations = {}

        # Loop through each epoch (pre and post) and stimulation level to calculate Pearson correlation coefficients
        for epoch in ['Pre', 'Post']: 
            for stim_level in stim_levels: 

                # Pool early and late means from all units for the current group, epoch, and stimulation level
                early_means = []
                late_means = []

                recording_names = EED.get_recording_names(group)
                for recording in recording_names:
                    cellid_names = EED.get_cellid_names(group, recording)
                    for cell_id in cellid_names:
                        early_mean = detailed_data[group][recording][cell_id][epoch][stim_level]['early_mean']
                        late_mean = detailed_data[group][recording][cell_id][epoch][stim_level]['late_mean']

                        if not np.isnan(early_mean) and not np.isnan(late_mean):
                            early_means.append(early_mean)
                            late_means.append(late_mean)
                        else:
                            removed_cell_ids.setdefault(group, {}).setdefault(recording, {}).setdefault(epoch, {}).setdefault(stim_level, []).append(cell_id)

                # Calculate the Pearson correlation coefficient
                if early_means and late_means:  # Ensure the lists are not empty
                    corr, _ = stats.pearsonr(early_means, late_means)
                else:
                    corr = None

                # Store the Pearson correlation coefficient in the dictionary
                group_correlations.setdefault(epoch, {}).setdefault(stim_level, corr)

        # Add the Pearson correlation coefficients for the current group to the dictionary
        pearson_correlations[group] = group_correlations

    return pearson_correlations, removed_cell_ids

# Use the function
pearson_correlations, removed_cell_ids = calculate_pearson_correlations(EED, pearson_correlations_global)


In [ ]:
pearson_correlations['Lmc_opsin']

In [ ]:
import numpy as np
import pandas as pd

def individual_unit_analysis(EED, group_name=None):
    """
    Perform a comprehensive individual unit analysis including descriptive statistics, 
    time series analysis, correlation analysis, visualization, response profiling, 
    and outlier identification.

    Parameters:
    EED (object): The object containing the electrophysiology data.
    group_name (str, optional): The name of the group to analyze. Defaults to None, in which case all groups are analyzed.

    Returns:
    dict: A dictionary containing the results of all the analyses for each unit.
    """
    
    # Get list of group names
    group_names = [group_name] if group_name else EED.group_names
    
    # Dictionary to store the analysis results for each group
    analysis_results = {}
    
    # Loop through each group
    for group in group_names:
        recording_names = EED.get_recording_names(group)
        
        # Dictionary to store the analysis results for all units in the current group
        group_analysis_results = {}
        
        # Loop through each recording
        for recording in recording_names:
            cellid_names = EED.get_cellid_names(group, recording)
            
            # Loop through each cell ID
            for cell_id in cellid_names:
                # Get the pre and post stim data
                data = EED.get_pre_post_data(group, recording, cell_id)
                
                # Define stimulation levels and pooled stimulation levels
                stim_levels = ['Zero', 'Low', 'Mid', 'Max', 'Pooled']
                
                # Dictionary to store the analysis results for the current unit
                unit_analysis_results = {'Recording': recording, 'CellID': cell_id}
                
                # Loop through each epoch (pre and post) and stimulation level to calculate descriptive statistics
                for epoch in ['Pre', 'Post']: #loop through the pre and post epochs
                    for stim_level in stim_levels: #loop through the stim levels
                        if stim_level == 'Pooled': #if the stim level is pooled, then use all stim levels
                            stim_indices = [1, 2, 3] #indices for the stim levels
                        else: #otherwise, use the stim level specified
                            stim_indices = [stim_levels.index(stim_level)] #create a list with the index of the stim level
                        
                        # Get spike trains for the current stimulation level
                        spiketrains = np.concatenate([data[epoch]['SpikeTrains_for_PSTHs'][i] for i in stim_indices], axis=0) #list comprehension to get the spike trains for the stim level by looping through the stim indices
                        
                        # Extract spike data for the early phase (0-50 ms post-stimulus)
                        early_phase = spiketrains[:, 500:550]  # Adjust indices as necessary
                        
                        # Extract spike data for the late phase (100-700 ms post-stimulus)
                        late_phase = spiketrains[:, 600:1200]  # Adjust indices as necessary
                        
                        # Calculate the total number of spikes in each trial during the early and late phases
                        early_response = early_phase.sum(axis=1) / early_phase.shape[0] # Normalize by the number of trials which is the number of rows
                        late_response = late_phase.sum(axis=1) / late_phase.shape[0] # Normalize by the number of trials which is the number of rows
                        
                        # Calculate descriptive statistics
                        early_mean = np.mean(early_response)
                        late_mean = np.mean(late_response)
                        early_median = np.median(early_response)
                        late_median = np.median(late_response)
                        early_std = np.std(early_response)
                        late_std = np.std(late_response) 
                        
                        # Calculate the mean of the standard deviations across trials
                        mean_std_early = np.mean(np.std(early_phase, axis=1))
                        mean_std_late = np.mean(np.std(late_phase, axis=1))
                        
                        # Step 2: Time Series Analysis
                        # Calculate the autocorrelation of the early and late responses
                        early_autocorr = np.correlate(early_response, early_response, mode='full')
                        late_autocorr = np.correlate(late_response, late_response, mode='full')
                        
                        # Step 3: Correlation Analysis
                        # Calculate the Pearson correlation coefficient between the early and late responses
                        corr, p_value = stats.pearsonr(early_response, late_response)

                        # Step 5: Creating Response Profiles
                        # Calculate the difference between the mean responses in the early and late phases
                        mean_response_difference = late_mean - early_mean
                        # Calculate the fold change relative to the early mean
                        fold_change = late_mean / early_mean if early_mean != 0 else np.nan
                        
 
                        # Store the data and descriptive statistics in the dictionary
                        unit_analysis_results[f'{epoch}_{stim_level}_early_phase'] = early_phase
                        unit_analysis_results[f'{epoch}_{stim_level}_late_phase'] = late_phase
                        unit_analysis_results[f'{epoch}_{stim_level}_early_response'] = early_response
                        unit_analysis_results[f'{epoch}_{stim_level}_late_response'] = late_response
                        unit_analysis_results[f'{epoch}_{stim_level}_early_mean'] = early_mean
                        unit_analysis_results[f'{epoch}_{stim_level}_late_mean'] = late_mean
                        unit_analysis_results[f'{epoch}_{stim_level}_early_median'] = early_median
                        unit_analysis_results[f'{epoch}_{stim_level}_late_median'] = late_median
                        unit_analysis_results[f'{epoch}_{stim_level}_early_std'] = early_std
                        unit_analysis_results[f'{epoch}_{stim_level}_late_std'] = late_std
                        unit_analysis_results[f'{epoch}_{stim_level}_mean_std_early'] = mean_std_early
                        unit_analysis_results[f'{epoch}_{stim_level}_mean_std_late'] = mean_std_late
                        
                        # Store the autocorrelation data in the dictionary
                        unit_analysis_results[f'{epoch}_{stim_level}_early_autocorr'] = early_autocorr
                        unit_analysis_results[f'{epoch}_{stim_level}_late_autocorr'] = late_autocorr
                        
                        # Store the Pearson correlation coefficient and the p-value in the dictionary
                        unit_analysis_results[f'{epoch}_{stim_level}_Pearson_Correlation'] = corr
                        unit_analysis_results[f'{epoch}_{stim_level}_P_value'] = p_value
                        
                        # Store the response profile metrics in the dictionary
                        unit_analysis_results[f'{epoch}_{stim_level}_mean_response_difference'] = mean_response_difference
                        unit_analysis_results[f'{epoch}_{stim_level}_fold_change'] = fold_change
                        
                # Add the analysis results for the current unit to the dictionary
                group_analysis_results[cell_id] = unit_analysis_results
        
        # Add the analysis results for the current group to the dictionary
        analysis_results[group] = group_analysis_results
        
    # Convert the nested dictionary to a pandas DataFrame for easy data manipulation and visualization
    analysis_results_df = pd.DataFrame.from_dict({(group_name, cell_id): analysis_results[group_name][cell_id] 
                                                  for group_name in analysis_results.keys() 
                                                  for cell_id in analysis_results[group_name].keys()},
                                                 orient='index')

    return  analysis_results, analysis_results_df




individual_unit_analysis_output, individual_unit_analysis_df_output = individual_unit_analysis(EED, group_name=None)



In [ ]:
individual_unit_analysis_df_output 

In [ ]:
individual_unit_analysis_output['Lmc_opsin']['cid102']['Post_Low_early_autocorr']

#now plot this data individual_unit_analysis_output['Lmc_opsin']['cid102']['Post_Low_early_autocorr']
plt.plot(individual_unit_analysis_output['Lmc_opsin']['cid93']['Post_Low_early_autocorr'])

In [ ]:
individual_unit_analysis_output['Lmc_opsin']['cid102']['Post_Low_Pearson_Correlation']
#now look at mid 
individual_unit_analysis_output['Lmc_opsin']['cid102']['Post_Mid_Pearson_Correlation']
#now look at max
individual_unit_analysis_output['Lmc_opsin']['cid102']['Post_Max_Pearson_Correlation'] 
#now look at zero
individual_unit_analysis_output['Lmc_opsin']['cid102']['Post_Zero_Pearson_Correlation']

#repeat this process for cid102 and print the pearson correlation coefficients for each stimulation level
print(individual_unit_analysis_output['Lmc_opsin']['cid102']['Post_Zero_Pearson_Correlation'])
print(individual_unit_analysis_output['Lmc_opsin']['cid102']['Post_Low_Pearson_Correlation'])
print(individual_unit_analysis_output['Lmc_opsin']['cid102']['Post_Mid_Pearson_Correlation'])
print(individual_unit_analysis_output['Lmc_opsin']['cid102']['Post_Max_Pearson_Correlation'])

#now print the mean for cid 102 for each stimulation level 
print(individual_unit_analysis_output['Lmc_opsin']['cid102']['Post_Zero_late_mean'])
print(individual_unit_analysis_output['Lmc_opsin']['cid102']['Post_Low_late_mean'])
print(individual_unit_analysis_output['Lmc_opsin']['cid102']['Post_Mid_late_mean'])
print(individual_unit_analysis_output['Lmc_opsin']['cid102']['Post_Max_late_mean'])



In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import matplotlib.colors as mcolors
import seaborn as sns

import plotly.graph_objects as go
from plotly.subplots import make_subplots


class NeuralDataAnalysis:
    def __init__(self, EED, group_name=None, early_phase_indices=(500, 550), late_phase_indices=(600, 1200)):
        self.EED = EED
        self.group_name = group_name
        self.early_phase_indices = early_phase_indices
        self.late_phase_indices = late_phase_indices
        self.analysis_results = None
        self.analysis_df = None
        
    def individual_unit_analysis(self, early_phase_indices=(500, 550), late_phase_indices=(600, 1200)):
        """
        Perform a comprehensive individual unit analysis including descriptive statistics, 
        time series analysis, correlation analysis, visualization, response profiling, 
        and outlier identification.

        Parameters:
        EED (object): The object containing the electrophysiology data.
        group_name (str, optional): The name of the group to analyze. Defaults to None, in which case all groups are analyzed.

        Returns:
        dict: A dictionary containing the results of all the analyses for each unit.
        """
        
        # Get list of group names
        group_names = [self.group_name] if self.group_name else self.EED.group_names
        
        # Dictionary to store the analysis results for each group
        analysis_results = {}
        
        # Loop through each group
        for group in group_names:
            recording_names = EED.get_recording_names(group)
            
            # Dictionary to store the analysis results for all units in the current group
            group_analysis_results = {}
            
            # Loop through each recording
            for recording in recording_names:
                cellid_names = EED.get_cellid_names(group, recording)
                
                # Loop through each cell ID
                for cell_id in cellid_names:
                    # Get the pre and post stim data
                    data = EED.get_pre_post_data(group, recording, cell_id)
                    
                    # Define stimulation levels and pooled stimulation levels
                    stim_levels = ['Zero', 'Low', 'Mid', 'Max', 'Pooled']
                    
                    # Dictionary to store the analysis results for the current unit
                    unit_analysis_results = {'Recording': recording, 'CellID': cell_id}
                    
                    # Loop through each epoch (pre and post) and stimulation level to calculate descriptive statistics
                    for epoch in ['Pre', 'Post']: #loop through the pre and post epochs
                        for stim_level in stim_levels: #loop through the stim levels
                            if stim_level == 'Pooled': #if the stim level is pooled, then use all stim levels
                                stim_indices = [1, 2, 3] #indices for the stim levels
                            else: #otherwise, use the stim level specified
                                stim_indices = [stim_levels.index(stim_level)] #create a list with the index of the stim level
                            
                            # Get spike trains for the current stimulation level
                            spiketrains = np.concatenate([data[epoch]['SpikeTrains_for_PSTHs'][i] for i in stim_indices], axis=0) #list comprehension to get the spike trains for the stim level by looping through the stim indices
                            
                            # Store the spike trains in the dictionary
                            unit_analysis_results[f'{epoch}_{stim_level}_spiketrains'] = spiketrains
                            
                            # Extract spike data for the early and late phases using the provided indices
                            early_phase = spiketrains[:, early_phase_indices[0]:early_phase_indices[1]]
                            late_phase = spiketrains[:, late_phase_indices[0]:late_phase_indices[1]]
                            
                            # Calculate the total number of spikes in each trial during the early and late phases
                            early_response = early_phase.sum(axis=1) / early_phase.shape[0] # Normalize by the number of trials which is the number of rows
                            late_response = late_phase.sum(axis=1) / late_phase.shape[0] # Normalize by the number of trials which is the number of rows
                            
                            # Calculate descriptive statistics
                            early_mean = np.mean(early_response)
                            late_mean = np.mean(late_response)
                            early_median = np.median(early_response)
                            late_median = np.median(late_response)
                            early_std = np.std(early_response)
                            late_std = np.std(late_response) 
                            
                            # Calculate the mean of the standard deviations across trials
                            mean_std_early = np.mean(np.std(early_phase, axis=1))
                            mean_std_late = np.mean(np.std(late_phase, axis=1))
                            
                            # Step 2: Time Series Analysis
                            # Calculate the autocorrelation of the early and late responses
                            early_autocorr = np.correlate(early_response, early_response, mode='full')
                            late_autocorr = np.correlate(late_response, late_response, mode='full')
                            
                            # Step 3: Correlation Analysis
                            # Calculate the Pearson correlation coefficient between the early and late responses
                            corr, p_value = stats.pearsonr(early_response, late_response)

                            # Step 5: Creating Response Profiles
                            # Calculate the difference between the mean responses in the early and late phases
                            mean_response_difference = late_mean - early_mean
                            
                            # Calculate the fold change relative to the early mean
                            fold_change = late_mean / early_mean if early_mean != 0 else np.nan
                            
                            # Calculate the fold change and percent change using the early window of the pre-epoch to normalize the late window of the post-epoch
                            if epoch == 'Post':
                                pre_epoch_early_mean = unit_analysis_results[f'Pre_{stim_level}_early_mean']
                                fold_change_normalized = late_mean / pre_epoch_early_mean if pre_epoch_early_mean != 0 else np.nan
                                unit_analysis_results[f'{epoch}_{stim_level}_fold_change_normalized'] = fold_change_normalized
                                percent_change_normalized = ((late_mean - pre_epoch_early_mean) / pre_epoch_early_mean) * 100 if pre_epoch_early_mean != 0 else np.nan
                                unit_analysis_results[f'{epoch}_{stim_level}_percent_change_normalized'] = percent_change_normalized

                            
    
                            # Store the data and descriptive statistics in the dictionary
                            unit_analysis_results[f'{epoch}_{stim_level}_early_phase'] = early_phase
                            unit_analysis_results[f'{epoch}_{stim_level}_late_phase'] = late_phase
                            unit_analysis_results[f'{epoch}_{stim_level}_early_response'] = early_response
                            unit_analysis_results[f'{epoch}_{stim_level}_late_response'] = late_response
                            unit_analysis_results[f'{epoch}_{stim_level}_early_mean'] = early_mean
                            unit_analysis_results[f'{epoch}_{stim_level}_late_mean'] = late_mean
                            unit_analysis_results[f'{epoch}_{stim_level}_early_median'] = early_median
                            unit_analysis_results[f'{epoch}_{stim_level}_late_median'] = late_median
                            unit_analysis_results[f'{epoch}_{stim_level}_early_std'] = early_std
                            unit_analysis_results[f'{epoch}_{stim_level}_late_std'] = late_std
                            unit_analysis_results[f'{epoch}_{stim_level}_mean_std_early'] = mean_std_early
                            unit_analysis_results[f'{epoch}_{stim_level}_mean_std_late'] = mean_std_late
                            
                            # Store the autocorrelation data in the dictionary
                            unit_analysis_results[f'{epoch}_{stim_level}_early_autocorr'] = early_autocorr
                            unit_analysis_results[f'{epoch}_{stim_level}_late_autocorr'] = late_autocorr
                            
                            # Store the Pearson correlation coefficient and the p-value in the dictionary
                            unit_analysis_results[f'{epoch}_{stim_level}_Pearson_Correlation'] = corr
                            unit_analysis_results[f'{epoch}_{stim_level}_P_value'] = p_value
                            
                            # Store the response profile metrics in the dictionary
                            unit_analysis_results[f'{epoch}_{stim_level}_mean_response_difference'] = mean_response_difference
                            unit_analysis_results[f'{epoch}_{stim_level}_fold_change'] = fold_change
                            
                    # Add the analysis results for the current unit to the dictionary
                    group_analysis_results[cell_id] = unit_analysis_results
            
            # Add the analysis results for the current group to the dictionary
            analysis_results[group] = group_analysis_results
            
        # Convert the nested dictionary to a pandas DataFrame for easy data manipulation and visualization
        analysis_results_df = pd.DataFrame.from_dict({(group_name, cell_id): analysis_results[group_name][cell_id] 
                                                    for group_name in analysis_results.keys() 
                                                    for cell_id in analysis_results[group_name].keys()},
                                                    orient='index')


        # Set the analysis_results attribute with the results dictionary
        self.analysis_results = analysis_results
        self.analysis_df = analysis_results_df
        
        # Return the DataFrame for inspection
        return analysis_results_df, analysis_results

    def rank_units_by_metric(self, group_name, column_name, ascending=True):
        """
        Rank units within a specified group based on a specified metric (e.g., Pearson correlation, fold change).

        Parameters:
        group_name (str): The name of the group to rank the units for.
        column_name (str): The name of the column (metric) to base the rankings on.
        ascending (bool): Whether to rank in ascending order. Defaults to True.

        Returns:
        pd.DataFrame: A DataFrame containing the unit IDs and their respective rankings.
        """
        
        
        group_data = self.analysis_df.loc[group_name].copy()
        
        # Rank the units based on the specified column
        group_data['Rank'] = group_data[column_name].rank(ascending=ascending)
        
        # Get a DataFrame with only the Cell IDs and their rankings
        ranking_df = group_data[['CellID', 'Rank']]
        
        return ranking_df 



    def plot_ranked_units(self, group_name, column_name, ascending=True):
        """
        Plot the ranked units within a specified group based on a specified metric (e.g., Pearson correlation, fold change).

        Parameters:
        group_name (str): The name of the group to rank the units for.
        column_name (str): The name of the column (metric) to base the rankings on.
        ascending (bool): Whether to rank in ascending order. Defaults to True.

        Returns:
        None: The function will plot the data but not return any values.
        """
        
        # Get the ranking data
        ranking_df = self.rank_units_by_metric(group_name, column_name, ascending)
        
        # Get the data to plot
        group_data = self.analysis_df.loc[group_name]
        
        # Merge the ranking data with the group data to get the values for the specified column
        plot_data = group_data.merge(ranking_df, on='CellID')
        
        # Sort the data based on the rank
        plot_data.sort_values(by='Rank', inplace=True)
        
        # Get a colormap
        cmap = plt.get_cmap('viridis')
        
        # Get the range of data values to set the color range
        data_min = plot_data[column_name].min()
        data_max = plot_data[column_name].max()
        
        # Create a normalized color range based on the data values
        norm = mcolors.Normalize(vmin=data_min, vmax=data_max)
        
        # Get a list of colors based on the normalized data values
        colors = cmap(norm(plot_data[column_name].values))
        
        # Plot the data with the colors
        plt.figure(figsize=(10, 6))
        plt.bar(plot_data['CellID'], plot_data[column_name], color=colors)
        
        # Set labels, title, etc.
        plt.xlabel('Cell ID')
        plt.ylabel(column_name)
        plt.title(f'Ranking of Units in {group_name} based on {column_name}')
        plt.xticks(rotation=90)
        plt.grid(axis='y')
        
        # Add a colorbar to show the color mapping
        sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
        sm.set_array([])
        plt.colorbar(sm)
        
        plt.show()
        
    def plot_mean_firing(self, stim_level, group_name=None):
        """
        This method plots the mean firing rates for individual units and the grand mean across units 
        during the early and late phases of the pre and post epochs. The early and late mean firing 
        rates are represented by lines connecting empty and filled circles, respectively. The grand 
        mean is represented by a red line.

        Parameters:
        stim_level (str): The stimulation level to consider ('Zero', 'Low', 'Mid', 'Max', or 'Pooled').
        group_name (str, optional): The name of the group to plot data for. If None, data for all groups are plotted.

        Returns:
        None: The method displays the plot but does not return any values.
        """
        
        # Ensure the analysis results DataFrame is available
        if self.analysis_df is None:
            print("Please run the individual_unit_analysis method first.")
            return

        # Get the list of group names
        group_names = [group_name] if group_name else self.EED.group_names
        
        # Create a 1x2 subplot with shared y-axis
        fig, axes = plt.subplots(1, 2, figsize=(8, 4), sharey=True)

        # Loop through each group
        for group in group_names:
            
            # Loop through each epoch (Pre and Post) to create the plots
            for i, epoch in enumerate(['Pre', 'Post']):
                early_means = self.analysis_df.loc[group, f'{epoch}_{stim_level}_early_mean']
                late_means = self.analysis_df.loc[group, f'{epoch}_{stim_level}_late_mean']
                
                unit_ids = self.analysis_df.loc[group].index.get_level_values(0).unique()
                
                # Lists to store the early and late means for all units
                early_means_list = []
                late_means_list = []
                
                # Loop through each unit to gather the early and late means
                for unit_id in unit_ids:
                    early_mean = early_means.loc[unit_id]
                    late_mean = late_means.loc[unit_id]
                    
                    early_means_list.append(early_mean)
                    late_means_list.append(late_mean)
                
                # Now you have lists of early and late means for all units, and you can plot lines connecting these values
                for j, unit_id in enumerate(unit_ids):
                    color = 'grey' if epoch == 'Pre' else 'blue'
                    axes[i].plot([1, 2], [early_means_list[j], late_means_list[j]], marker='o', markersize=5, markerfacecolor=color, markeredgecolor=color, linestyle='-', color=color)
                    axes[i].plot(1, early_means_list[j], marker='o', markersize=5, markerfacecolor='white', markeredgecolor=color, linestyle='-', color=color)
                
                # Calculate the grand mean for all units
                grand_mean_early = np.mean(early_means_list)
                grand_mean_late = np.mean(late_means_list)
                
                # Plot the grand mean
                axes[i].plot([1, 2], [grand_mean_early, grand_mean_late], marker='o', markersize=7, markerfacecolor='none', color='red', linewidth=2, label='Grand Mean')
                
                # Set the x-ticks labels and title
                axes[i].set_xticks([1, 2])
                axes[i].set_xticklabels(['Early-Phase', 'Late-Phase'])
                axes[i].set_title(f'{epoch} Epoch with {stim_level} Stimulation')
                axes[i].set_xlabel('Phase')
                axes[i].set_ylabel('Mean Firing Rate')
            
            # Display a legend in the second plot
            axes[1].legend(loc='upper left', bbox_to_anchor=(1, 1), ncol=1)
            
            # Adjust the layout to prevent overlapping
            plt.tight_layout()
            
            # Display the plot
            plt.show()
            
    def plot_standard_deviation(self, stim_level, group_name=None):
        """
        This method plots the standard deviation of firing rates for individual units and the grand mean across units 
        during the early and late phases of the pre and post epochs. The early and late standard deviations 
        are represented by lines connecting empty and filled circles, respectively. The grand mean is represented 
        by a red line for the pre epoch and a green line for the post epoch.

        Parameters:
        stim_level (str): The stimulation level to consider ('Zero', 'Low', 'Mid', 'Max', or 'Pooled').
        group_name (str, optional): The name of the group to plot data for. If None, data for all groups are plotted.

        Returns:
        None: The method displays the plot but does not return any values.
        """
        
        # Ensure the analysis results DataFrame is available
        if self.analysis_df is None:
            print("Please run the individual_unit_analysis method first.")
            return

        # Get the list of group names
        group_names = [group_name] if group_name else self.EED.group_names
        
        # Create a 1x2 subplot with shared y-axis
        fig, axes = plt.subplots(1, 2, figsize=(8, 4), sharey=True)
        
        # Loop through each group
        for group in group_names:
            
            # Loop through each epoch (Pre and Post) to create the plots
            for i, epoch in enumerate(['Pre', 'Post']):
                early_stds = self.analysis_df.loc[group, f'{epoch}_{stim_level}_early_std']
                late_stds = self.analysis_df.loc[group, f'{epoch}_{stim_level}_late_std']
                
                unit_ids = self.analysis_df.loc[group].index.get_level_values(0).unique()
                
                # Lists to store the early and late standard deviations for all units
                early_stds_list = []
                late_stds_list = []
                
                # Loop through each unit to gather the early and late standard deviations
                for unit_id in unit_ids:
                    early_std = early_stds.loc[unit_id]
                    late_std = late_stds.loc[unit_id]
                    
                    early_stds_list.append(early_std)
                    late_stds_list.append(late_std)
                
                # Now you have lists of early and late standard deviations for all units, and you can plot lines connecting these values
                for j, unit_id in enumerate(unit_ids):
                    color = 'grey' if epoch == 'Pre' else 'blue'
                    axes[i].plot([1, 2], [early_stds_list[j], late_stds_list[j]], marker='o', markersize=5, markerfacecolor=color, markeredgecolor=color, linestyle='-', color=color)
                    axes[i].plot(1, early_stds_list[j], marker='o', markersize=5, markerfacecolor='white', markeredgecolor=color, linestyle='-', color=color)
                    

                
                # Calculate the grand mean for all units
                grand_mean_early = np.mean(early_stds_list)
                grand_mean_late = np.mean(late_stds_list)
                
                # Plot the grand mean
                axes[i].plot([1, 2], [grand_mean_early, grand_mean_late], marker='o', markersize=7, markerfacecolor='none', color='red', linewidth=2, label='Grand Mean')
                
                # Set the x-ticks labels and title
                # And update the x-ticks and labels as follows:
                axes[i].set_xticks([1, 2])
                axes[i].set_xticklabels(['Early-Phase', 'Late-Phase'])
                axes[i].set_title(f'{epoch} Epoch with {stim_level} Stimulation')
                axes[i].set_xlabel('Phase')
                axes[i].set_ylabel('Standard Deviation of Firing Rate')
            
            # Display a legend in the second plot
            axes[1].legend(loc='upper left', bbox_to_anchor=(1, 1), ncol=1)
            
            # Adjust the layout to prevent overlapping
            plt.tight_layout()
            
            # Display the plot
            plt.show()
    def plot_fold_change(self, stim_level, group_name=None):
        """
        This method plots the fold change of firing rates for individual units and the grand mean across units 
        during the early and late phases of the pre and post epochs. The fold changes 
        are represented by lines connecting circles, with the baseline represented as 1. The grand mean is represented 
        by a red line.

        Parameters:
        stim_level (str): The stimulation level to consider ('Zero', 'Low', 'Mid', 'Max', or 'Pooled').
        group_name (str, optional): The name of the group to plot data for. If None, data for all groups are plotted.

        Returns:
        None: The method displays the plot but does not return any values.
        """
        
        # Ensure the analysis results DataFrame is available
        if self.analysis_df is None:
            print("Please run the individual_unit_analysis method first.")
            return

        # Get the list of group names
        group_names = [group_name] if group_name else self.EED.group_names
        
        # Create a 1x2 subplot with shared y-axis
        fig, axes = plt.subplots(1, 2, figsize=(8, 4), sharey=True)
        
        # Loop through each group
        for group in group_names:
            
            # Loop through each epoch (Pre and Post) to create the plots
            for i, epoch in enumerate(['Pre', 'Post']):
                
                fold_changes = self.analysis_df.loc[group, f'{epoch}_{stim_level}_fold_change']
                unit_ids = self.analysis_df.loc[group].index.get_level_values(0).unique()
                
                # Lists to store the fold changes for all units
                fold_changes_list = []
                
                # Loop through each unit to gather the fold changes
                for unit_id in unit_ids:
                    fold_change = fold_changes.loc[unit_id]
                    fold_changes_list.append(fold_change)
                
                # Now you have a list of fold changes for all units, and you can plot lines connecting these values
                for j, unit_id in enumerate(unit_ids):
                    color = 'grey' if epoch == 'Pre' else 'blue'
                    axes[i].plot([1, 2], [1, fold_changes_list[j]], marker='o', markersize=5, markerfacecolor=color, markeredgecolor=color, linestyle='-', color=color)
                    
                # Calculate the grand mean for all units
                grand_mean_baseline = 1  # The baseline is 1 for fold change
                grand_mean_fold_change = np.mean(fold_changes_list)
                
                # Plot the grand mean
                axes[i].plot([1, 2], [grand_mean_baseline, grand_mean_fold_change], marker='o', markersize=7, markerfacecolor='none', color='red', linewidth=2, label='Grand Mean')
                
                # Set the x-ticks labels and title
                axes[i].set_xticks([1, 2])
                axes[i].set_xticklabels(['Baseline', 'Fold Change'])
                axes[i].set_title(f'{epoch} Epoch with {stim_level} Stimulation')
                axes[i].set_xlabel('Phase')
                axes[i].set_ylabel('Fold Change of Firing Rate')
            
            # Display a legend in the second plot
            axes[1].legend(loc='upper left', bbox_to_anchor=(1, 1), ncol=1)
            
            # Adjust the layout to prevent overlapping
            plt.tight_layout()
            
            # Display the plot
            plt.show()

    def compare_percent_changes_between_groups(self, group1_name, group2_name):
        # Ensure the analysis_df attribute is not None
        if self.analysis_df is None:
            raise ValueError("The analysis_df attribute is None. Please run the individual_unit_analysis method first.")

        # Define the stimulation levels
        stim_levels = ['Zero', 'Low', 'Mid', 'Max', 'Pooled']

        # Initialize an empty list to store the results of the t-tests
        t_test_results = []

        # Loop through each stimulation level to perform the t-tests
        for stim_level in stim_levels:
            # Get the percent changes for the current stimulation level for each group
            group1_data = self.analysis_df.loc[(group1_name, slice(None)), :][f'Post_{stim_level}_percent_change_normalized'].values
            group2_data = self.analysis_df.loc[(group2_name, slice(None)), :][f'Post_{stim_level}_percent_change_normalized'].values

            # Perform an independent t-test
            t_stat, p_val = stats.ttest_ind(group1_data, group2_data, nan_policy='omit')

            # Store the results in the list
            t_test_results.append({'Stimulation Level': stim_level, 't-statistic': t_stat, 'p-value': p_val})

        # Create a DataFrame from the t-test results
        #t_test_results_df = pd.DataFrame(t_test_results)

        # Print the t-test results
        #print(t_test_results_df)

        # Create a bar plot to visualize the p-values
        #sns.set(style="whitegrid")
        #plt.figure(figsize=(10, 6))
        #sns.barplot(x='Stimulation Level', y='p-value', data=t_test_results_df)
        #plt.axhline(y=0.05, color='r', linestyle='--')  # Add a line to indicate the 0.05 significance level
        #plt.title('Significance of Percent Changes (Group1 vs. Group2)')
        #plt.ylabel('p-value')
        #plt.xlabel('Stimulation Level')
        #plt.show()
        
        # Create a dictionary of p-values indexed by stimulation level
        p_values = {result['Stimulation Level']: result['p-value'] for result in t_test_results}
    
        # Return the dictionary of p-values at the end of the method
        return p_values
        
            
            
    def plot_fold_change_plotly(self, stim_level, group_name=None):
        """
        This method plots the fold change of firing rates for individual units and the grand mean across units 
        during the baseline and fold change phases of the pre and post epochs using Plotly for interactive visualization. 
        The baseline and fold change values are represented by lines connecting markers at x=1 and x=2, respectively. 
        The grand mean is represented by a red line for the pre epoch and a green line for the post epoch.

        Parameters:
        stim_level (str): The stimulation level to consider ('Zero', 'Low', 'Mid', 'Max', or 'Pooled').
        group_name (str, optional): The name of the group to plot data for. If None, data for all groups are plotted.

        Returns:
        None: The method displays the plot but does not return any values.
        """
        
        # Ensure the analysis results DataFrame is available
        if self.analysis_df is None:
            print("Please run the individual_unit_analysis method first.")
            return

        # Get the list of group names
        group_names = [group_name] if group_name else self.EED.group_names
        
        # Loop through each group
        for group in group_names:
            
            # Create a Plotly figure
            fig = go.Figure()
            
            # Loop through each epoch (Pre and Post) to create the plots
            for i, epoch in enumerate(['Pre', 'Post']):
                fold_changes = self.analysis_df.loc[group, f'{epoch}_{stim_level}_fold_change']
                
                unit_ids = self.analysis_df.loc[group].index.get_level_values(0).unique()
                
                # Lists to store the baseline and fold change values for all units
                baseline_list = [1] * len(unit_ids)
                fold_changes_list = []
                
                # Loop through each unit to gather the baseline and fold change values
                for unit_id in unit_ids:
                    fold_change = fold_changes.loc[unit_id]
                    fold_changes_list.append(fold_change)
                
                # Now you have lists of baseline and fold change values for all units, and you can plot lines connecting these values
                for j, unit_id in enumerate(unit_ids):
                    color = 'grey' if epoch == 'Pre' else 'blue'
                    fig.add_trace(go.Scatter(x=[1, 2], y=[1, fold_changes_list[j]], mode='lines+markers', marker=dict(color=color), line=dict(color=color), name=f'{unit_id} {epoch}', hovertemplate=f'Unit ID: {unit_id}<br>Phase: '+'%{x}<br>Fold Change: '+'%{y}'))

                # Calculate the grand mean for all units
                grand_mean_baseline = np.nanmean(baseline_list)
                grand_mean_fold_change = np.nanmean(fold_changes_list)
                
                
                # Plot the grand mean
                color = 'red' if epoch == 'Pre' else 'green'
                fig.add_trace(go.Scatter(x=[1, 2], y=[grand_mean_baseline, grand_mean_fold_change], mode='lines+markers', marker=dict(color=color), line=dict(color=color, width=2), name=f'Grand Mean {epoch}'))
            
            # Set the x-ticks labels and title            
            fig.update_layout(
                xaxis=dict(tickvals=[1, 2], ticktext=['Baseline', 'Fold Change']),
                title=f'Fold Change with {stim_level} Stimulation for group {group}',
                xaxis_title='Phase',
                yaxis_title='Fold Change',
                template='plotly_white',  # Change the template here
                width=800,  # Set the width of the plot (in pixels)
                height=600,  # Set the height of the plot (in pixels)
            )
            # Get the total number of traces (2 grand means + number of units for pre and post)
            num_traces = len(fig.data)

            # Define the dropdown buttons with dynamic visibility lists
            buttons = [
                dict(
                    args=[{'visible': [True] * num_traces}],  # Show all traces (both pre and post)
                    label='Both',
                    method='update'
                ),
                dict(
                    args=[{'visible': [True if i < num_traces // 2 else False for i in range(num_traces)]}],  # Show only pre traces
                    label='Pre',
                    method='update'
                ),
                dict(
                    args=[{'visible': [False if i < num_traces // 2 else True for i in range(num_traces)]}],  # Show only post traces
                    label='Post',
                    method='update'
                )
            ]

            # Add the dropdown buttons to the figure
            fig.update_layout(
                updatemenus=[
                    dict(
                        x=1.15,
                        y=1.2,
                        xanchor='left',
                        yanchor='top',
                        buttons=buttons
                    )
                ]
            )
            # Display the plot
            fig.show()

    def plot_standard_deviation_plotly(self, stim_level, group_name=None, data_type='both'):
            # Ensure the analysis results DataFrame is available
            if self.analysis_df is None:
                print("Please run the individual_unit_analysis method first.")
                return

            # Get the list of group names
            group_names = [group_name] if group_name else self.EED.group_names
            
            # Loop through each group
            for group in group_names:
                
                # Create a Plotly figure
                fig = go.Figure()
                
                # Loop through each epoch (Pre and Post) to create the plots
                for i, epoch in enumerate(['Pre', 'Post']):
                    
                    # Get the standard deviations for the early and late phases
                    early_std_devs = self.analysis_df.loc[group, f'{epoch}_{stim_level}_early_std']
                    late_std_devs = self.analysis_df.loc[group, f'{epoch}_{stim_level}_late_std']
                    
                    unit_ids = self.analysis_df.loc[group].index.get_level_values(0).unique()
                    
                    # Loop through each unit to gather the standard deviation values for early and late phases
                    for unit_id in unit_ids:
                        early_std_dev = early_std_devs.loc[unit_id]
                        late_std_dev = late_std_devs.loc[unit_id]
                        
                        color = 'grey' if epoch == 'Pre' else 'blue'
                        fig.add_trace(go.Scatter(x=['Early', 'Late'], y=[early_std_dev, late_std_dev], mode='lines+markers', marker=dict(color=color), line=dict(color=color), name=f'{unit_id} {epoch}', hovertemplate=f'Unit ID: {unit_id}<br>Phase: '+'%{x}<br>Std Dev: '+'%{y}'))
                    
                    # Get the mean standard deviations across units for early and late phases
                    mean_early_std_dev = np.nanmean(early_std_devs)
                    mean_late_std_dev = np.nanmean(late_std_devs)

                    color = 'red' if epoch == 'Pre' else 'green'
                    fig.add_trace(go.Scatter(x=['Early', 'Late'], y=[mean_early_std_dev, mean_late_std_dev], mode='lines+markers', marker=dict(color=color), line=dict(color=color, width=2), name=f'Mean Std Dev {epoch}'))
            
            # Set the x-ticks labels and title            
            fig.update_layout(
                xaxis=dict(tickvals=[0, 1], ticktext=['Early', 'Late']),
                title=f'Standard Deviation with {stim_level} Stimulation for group {group}',
                xaxis_title='Phase',
                yaxis_title='Standard Deviation',
                template='plotly_white',
                width=800,
                height=600,
            )
            
            # Get the total number of traces (2 grand means + number of units for pre and post)
            num_traces = len(fig.data)

            # Define the dropdown buttons with dynamic visibility lists
            buttons = [
                dict(
                    args=[{'visible': [True] * num_traces}],  # Show all traces (both pre and post)
                    label='Both',
                    method='update'
                ),
                dict(
                    args=[{'visible': [True if i < num_traces // 2 else False for i in range(num_traces)]}],  # Show only pre traces
                    label='Pre',
                    method='update'
                ),
                dict(
                    args=[{'visible': [False if i < num_traces // 2 else True for i in range(num_traces)]}],  # Show only post traces
                    label='Post',
                    method='update'
                )
            ]

            # Add the dropdown buttons to the figure
            fig.update_layout(
                updatemenus=[           
                    dict(
                        x=1.15,
                        y=1.2,
                        xanchor='left',
                        yanchor='top',
                        buttons=buttons
                    )
                ]
            )
            
            # Display the plot
            fig.show()
            
            
    def plot_mean_firing_rate_plotly(self, stim_level, group_name=None):
        # Ensure the analysis results DataFrame is available
        if self.analysis_df is None:
            print("Please run the individual_unit_analysis method first.")
            return

        # Get the list of group names
        group_names = [group_name] if group_name else self.EED.group_names
        
        # Loop through each group
        for group in group_names:
            
            # Create a Plotly figure
            fig = go.Figure()
            
            # Loop through each epoch (Pre and Post) to create the plots
            for i, epoch in enumerate(['Pre', 'Post']):
                
                # Get the mean firing rates for the early and late phases
                early_mean_rates = self.analysis_df.loc[group, f'{epoch}_{stim_level}_early_mean']
                late_mean_rates = self.analysis_df.loc[group, f'{epoch}_{stim_level}_late_mean']
                
                unit_ids = self.analysis_df.loc[group].index.get_level_values(0).unique()
                
                # Loop through each unit to gather the mean firing rates for early and late phases
                for unit_id in unit_ids:
                    early_mean_rate = early_mean_rates.loc[unit_id]
                    late_mean_rate = late_mean_rates.loc[unit_id]
                    
                    color = 'grey' if epoch == 'Pre' else 'blue'
                    fig.add_trace(go.Scatter(x=['Early', 'Late'], y=[early_mean_rate, late_mean_rate], mode='lines+markers', marker=dict(color=color), line=dict(color=color), name=f'{unit_id} {epoch}', hovertemplate=f'Unit ID: {unit_id}<br>Phase: '+'%{x}<br>Mean Rate: '+'%{y}'))
                
                # Get the grand mean firing rates across units for early and late phases
                grand_mean_early = np.nanmean(early_mean_rates)
                grand_mean_late = np.nanmean(late_mean_rates)

                color = 'red' if epoch == 'Pre' else 'green'
                fig.add_trace(go.Scatter(x=['Early', 'Late'], y=[grand_mean_early, grand_mean_late], mode='lines+markers', marker=dict(color=color), line=dict(color=color, width=2), name=f'Grand Mean {epoch}'))
            
            # Set the x-ticks labels and title            
            fig.update_layout(
                xaxis=dict(tickvals=[0, 1], ticktext=['Early', 'Late']),
                title=f'Mean Firing Rate with {stim_level} Stimulation for group {group}',
                xaxis_title='Phase',
                yaxis_title='Mean Firing Rate',
                template='plotly_white',
                width=800,
                height=600,
            )
            
            # Get the total number of traces (2 grand means + number of units for pre and post)
            num_traces = len(fig.data)

            # Define the dropdown buttons with dynamic visibility lists
            buttons = [
                dict(
                    args=[{'visible': [True] * num_traces}],  # Show all traces (both pre and post)
                    label='Both',
                    method='update'
                ),
                dict(
                    args=[{'visible': [True if i < num_traces // 2 else False for i in range(num_traces)]}],  # Show only pre traces
                    label='Pre',
                    method='update'
                ),
                dict(
                    args=[{'visible': [False if i < num_traces // 2 else True for i in range(num_traces)]}],  # Show only post traces
                    label='Post',
                    method='update'
                )
            ]

            # Add the dropdown buttons to the figure
            fig.update_layout(
                updatemenus=[
                    dict(
                        x=1.15,
                        y=1.2,
                        xanchor='left',
                        yanchor='top',
                        buttons=buttons
                    )
                ]
            )
            
            # Display the plot
            fig.show()
            

    def plot_group_responses(self, group_name):
        # Ensure the analysis results DataFrame is available
        if self.analysis_df is None:
            print("Please run the individual_unit_analysis method first.")
            return

        # Get the unique unit IDs for the specified group
        unit_ids = self.analysis_df.loc[group_name].index.get_level_values(0).unique()

        # Create a Plotly figure
        fig = go.Figure()

        # Loop through each unit ID to add a scatter plot trace for each combination of epoch, stim level, and phase
        for unit_id in unit_ids:
            for epoch in ['Pre', 'Post']:
                for stim_level in ['Zero', 'Low', 'Mid', 'Max', 'Pooled']:
                    for phase in ['early', 'late']:
                        response = self.analysis_df.loc[(group_name, unit_id), f'{epoch}_{stim_level}_{phase}_response']
                        fig.add_trace(go.Scatter(x=list(range(len(response))), y=response, mode='markers', name=f'{unit_id} {epoch} {stim_level} {phase}'))

        # Create a dictionary to store the current visibility state of each option
        current_visibility = {'epoch': ['Pre', 'Post'], 'stim_level': ['Zero', 'Low', 'Mid', 'Max', 'Pooled'], 'phase': ['early', 'late']}

        # Define a function to update the visibility of the traces based on the selected options
        def update_visibility(option_type, option_value):
            if option_value in ['Both', 'All']:
                current_visibility[option_type] = ['Pre', 'Post'] if option_type == 'epoch' else ['Zero', 'Low', 'Mid', 'Max', 'Pooled'] if option_type == 'stim_level' else ['early', 'late']
            else:
                current_visibility[option_type] = [option_value]
            return [all(any(option in trace.name for option in current_visibility[option_type]) for option_type in current_visibility) for trace in fig.data]

        # Define dropdown menus to control visibility of traces
        epoch_buttons = [
            dict(label=epoch, method="update", args=[{"visible": update_visibility('epoch', epoch)}]) for epoch in ['Pre', 'Post', 'Both']
        ]
        stim_level_buttons = [
            dict(label=stim_level, method="update", args=[{"visible": update_visibility('stim_level', stim_level)}]) for stim_level in ['Zero', 'Low', 'Mid', 'Max', 'Pooled', 'All']
        ]
        phase_buttons = [
            dict(label=phase, method="update", args=[{"visible": update_visibility('phase', phase)}]) for phase in ['early', 'late', 'Both']
        ]

        # Add dropdown menus to the figure
        fig.update_layout(
            updatemenus=[
                dict(
                    x=1.15,
                    y=1.2,
                    xanchor='left',
                    yanchor='top',
                    buttons=epoch_buttons
                ),
                dict(
                    x=1.15,
                    y=1.1,
                    xanchor='left',
                    yanchor='top',
                    buttons=stim_level_buttons
                ),
                dict(
                    x=1.15,
                    y=1.0,
                    xanchor='left',
                    yanchor='top',
                    buttons=phase_buttons
                )
            ]
        )

        # Display the plot
        fig.show()
        
    def plot_group_responses_histogram(self, group_name, nbinsx=100):
        """
        Plot a histogram of the responses for a specified group.

        Parameters:
        group_name (str): The name of the group to plot.
        nbinsx (int, optional): The number of bins for the histogram. Defaults to 100.

        Returns:
        None
        """
        
        # Ensure the analysis results DataFrame is available
        if self.analysis_df is None:
            print("Please run the individual_unit_analysis method first.")
            return

        # Define the epochs, stimulation levels, and phases
        epochs = ['Pre', 'Post']
        stim_levels = ['Zero', 'Low', 'Mid', 'Max', 'Pooled']
        phases = ['early', 'late']

        # Define a list of colors to use for the different traces
        colors = ['red', 'blue', 'green', 'purple', 'orange', 'pink', 'brown', 'yellow', 'grey', 'cyan']

        # Create a Plotly figure
        fig = go.Figure()

        # Loop through each epoch, stimulation level, and phase to create the plots
        from itertools import product
        for i, (epoch, stim_level, phase) in enumerate(product(epochs, stim_levels, phases)):
            response_data = self.analysis_df.loc[group_name, f'{epoch}_{stim_level}_{phase}_response'].values
            
            fig.add_trace(go.Histogram(
                x=response_data,
                nbinsx=nbinsx,  # Use the parameter to set the number of bins
                name=f'{epoch} {stim_level} {phase}',
                marker_color=colors[i % len(colors)],
                hoverinfo='x+y',
                hovertemplate='Count: %{y}<br>Response: %{x}<extra></extra>',
            ))

        # Define the dropdown buttons with dynamic visibility lists
        buttons = [
            dict(
                args=[{'visible': [True] * len(fig.data)}], 
                label='All',
                method='update'
            )
        ]
        for i, (epoch, stim_level, phase) in enumerate(product(epochs, stim_levels, phases)):
            visible = [False] * len(fig.data)
            visible[i] = True
            buttons.append(
                dict(
                    args=[{'visible': visible}], 
                    label=f'{epoch} {stim_level} {phase}',
                    method='update'
                )
            )

        # Add the dropdown buttons to the figure
        fig.update_layout(
            updatemenus=[
                dict(
                    x=1.15,
                    y=1.2,
                    xanchor='left',
                    yanchor='top',
                    buttons=buttons
                )
            ]
        )
        
        # Set the x-ticks labels and title            
        fig.update_layout(
            title=f'Response Histogram for {group_name} group',
            xaxis_title='Response',
            xaxis=dict(
                showticklabels=False,  # Hide x-axis labels
                showgrid=False  # Remove x-axis grid lines
            ),
            yaxis_title='Count',
            template='plotly_white',
            width=800,
            height=600,
        )
        
        # Display the plot
        fig.show()

    def plot_group_responses_histogramsubplots(self, group_name, nbinsx=20):
        # Define the stimulation levels and phases
        stim_levels = ['Zero', 'Low', 'Mid', 'Max', 'Pooled']
        phases = ['early', 'late']
        epochs = ['Pre', 'Post']

        # Get the unique unit IDs for the specified group
        unit_ids = self.analysis_df.loc[(group_name, slice(None)), :].index.get_level_values(1).unique()
        all_unit_ids = list(unit_ids)

        # Create a subplot with 2 rows and 2 columns
        fig = make_subplots(rows=2, cols=2, subplot_titles=('Pre Early', 'Pre Late', 'Post Early', 'Post Late'))

        # Function to get pooled data for the current selection of units
        def get_pooled_data(epoch, phase, stim_level, visible_unit_ids):
            pooled_data = []
            for unit_id in visible_unit_ids:
                data = self.analysis_df.loc[(group_name, unit_id), f'{epoch}_{stim_level}_{phase}_response']
                pooled_data.extend(data)
            return pooled_data

        # Initially, all units are visible
        visible_unit_ids = all_unit_ids.copy()

        # Loop through each condition to plot the histograms
        for i, epoch in enumerate(epochs):
            for j, phase in enumerate(phases):
                data = get_pooled_data(epoch, phase, 'Pooled', visible_unit_ids)
                fig.add_trace(go.Histogram(x=data, nbinsx=nbinsx, name='Pooled Data'), row=i+1, col=j+1)

        # Create buttons to include or remove units
        buttons = [
            dict(
                args=[
                    {
                        'x': [
                            get_pooled_data(epoch, phase, 'Pooled', [unit_id for unit_id in visible_unit_ids if unit_id != uid]) 
                            for epoch in epochs 
                            for phase in phases
                        ]
                    }, 
                    list(range(4))
                ],
                label='Unit ' + str(uid),
                method='restyle'
            )
            for uid in all_unit_ids
        ]

        stim_dropdown = [
            dict(
                args=[
                    {
                        'x': [
                            get_pooled_data(epoch, phase, stim_level, visible_unit_ids) 
                            for epoch in epochs 
                            for phase in phases
                        ]
                    }, 
                    list(range(4))
                ],
                label=stim_level,
                method='restyle'
            )
            for stim_level in stim_levels
        ]

        # Update layout to include the buttons and other details
        fig.update_layout(
            updatemenus=[
                dict(
                    buttons=stim_dropdown,
                    direction="down",
                    pad={"r": 10, "t": 10},
                    showactive=True,
                    x=0.24,
                    xanchor="left",
                    y=1.15,
                    yanchor="top"
                ),
                *[
                    dict(
                        buttons=[button],
                        direction="down",
                        pad={"r": 10, "t": 10},
                        showactive=True,
                        x=0.17,
                        xanchor="left",
                        y=1.15 - i * 0.05,  # Adjust the y position for each button to avoid overlap
                        yanchor="top"
                    )
                    for i, button in enumerate(buttons)
                ]
            ]
        )

        # Display the plot
        fig.show()



            
    def remove_cell_ids(self, group_name, cell_ids_to_remove):
        """
        This method removes specific cell IDs from a specified group in the EED class.

        Parameters:
        group_name (str): The name of the group from which to remove cell IDs.
        cell_ids_to_remove (list): A list of cell IDs to remove from the group.

        Returns:
        None
        """
        
        # Check if the analysis DataFrame is available
        if self.analysis_df is None:
            print("The analysis DataFrame is not available.")
            return
        
        # Check if the group name is valid
        if group_name not in self.EED.group_names:
            print(f"The group name '{group_name}' does not exist.")
            return

        # Remove the specified cell IDs from the specified group
        self.analysis_df = self.analysis_df.drop(index=cell_ids_to_remove, level=1)

        print(f"Removed cell IDs {cell_ids_to_remove} from group '{group_name}'.")





analysis_instance = NeuralDataAnalysis(EED, group_name=None, early_phase_indices= (500, 525) , late_phase_indices= (526, 800))

result_df, results_dict = analysis_instance.individual_unit_analysis()

In [ ]:
from scipy import stats
import numpy as np
import time
import warnings
from scipy.stats import ConstantInputWarning

warnings.filterwarnings("ignore", category=ConstantInputWarning)


def find_optimal_late_phase_range(analysis_instance, group1_name, group2_name, min_start=550, max_end=1200, min_step=1):
    
    # Define the stimulation levels
    stim_levels = ['Zero', 'Low', 'Mid', 'Max']
    
    # Initialize variables to store the optimal range, step and minimum p-value before entering the step loop
    optimal_range = (min_start, min_start + min_step)
    optimal_step = min_step
    min_p_value = 0.01  # Set the initial minimum p-value to a high value

    max_step = max_end - min_start

    # Loop through different step sizes from the maximum to the minimum
    for step in range(max_step, min_step - 1, -min_step):
       
        # Calculate the number of iterations
        num_start_iterations = ((max_end - step - min_start) // step) + 1
        num_end_iterations = ((max_end - (min_start + step)) // step) + 1
        total_iterations = num_start_iterations * num_end_iterations

        # Print the total number of iterations
        print(f"Total number of iterations for step size {step}: {total_iterations}")

        # Start a timer to estimate the time to run
        start_time = time.time()

        # Run a few iterations to estimate the average time per iteration
        for i in range(5):
            analysis_instance.individual_unit_analysis(early_phase_indices=(500, 550), late_phase_indices=(min_start, min_start + step))
            analysis_instance.compare_percent_changes_between_groups(group1_name, group2_name)
        avg_time_per_iteration = (time.time() - start_time) / 5

        # Print the estimated time to run
        estimated_time_to_run = avg_time_per_iteration * total_iterations
        print(f"Estimated time to run: {estimated_time_to_run / 60:.2f} minutes")
        
        # Initialize a counter before entering your loop
        iteration_counter = 0

        # Loop through different start and end indices for the late phase
        # Loop through different start and end indices for the late phase
        for start in range(min_start, max_end - step, step):
            for end in range(start + step, max_end, step):
                # Inside your loop, increment the counter with each iteration
                iteration_counter += 1
                
                # Update the late phase indices in the individual_unit_analysis method
                analysis_instance.individual_unit_analysis(early_phase_indices=(500, 550), late_phase_indices=(start, end))
                
                # Get the p-values for the current range of late phase indices
                p_values = analysis_instance.compare_percent_changes_between_groups(group1_name, group2_name)
                
                # Get the maximum p-value across the specified stimulation levels
                max_p_value = max([p_values[stim] for stim in stim_levels[1:-1]]) # Exclude the 'Zero' and 'Pooled' stimulation levels

                    
                if max_p_value < min_p_value or (max_p_value < 0.05 and (end - start) > (optimal_range[1] - optimal_range[0])):
                    min_p_value = max_p_value
                    optimal_range = (start, end)
                    optimal_step = step  # Store the current step size as the optimal step size
   
                
                # Use an if statement with the modulo operator to print the p-values every 100 iterations
                if iteration_counter % 100 == 0:
                    print(f"Iteration {iteration_counter}: {dict(zip(stim_levels, [p_values[stim] for stim in stim_levels]))}")

    # Return the optimal range, step and minimum p-value
    return optimal_range, optimal_step, min_p_value

# Usage
optimal_range, optimal_step, min_p_value = find_optimal_late_phase_range(analysis_instance, 'Lmc_opsin', 'Lmc_noopsin')
print(f"Optimal range: {optimal_range}, Optimal step: {optimal_step}, Minimum p-value: {min_p_value}")



In [ ]:
import matplotlib.pyplot as plt
import pandas as pd


def plot_and_rank_units(analysis_instance, epoch, stim_level):
    # Get the spike trains data
    spiketrains = analysis_instance.analysis_df[f'{epoch}_{stim_level}_spiketrains']

    # Calculate the total number of spikes for each unit ID across all trials
    total_spikes_per_unit = spiketrains.apply(lambda x: x.sum())

    # Rank the unit IDs based on the total number of spikes
    ranked_units = total_spikes_per_unit.sort_values(ascending=False)

    # Create a bar plot to visualize the ranked unit IDs
    plt.figure(figsize=(10, 6))
    ranked_units.plot(kind='bar')
    
    # Set plot labels and title
    plt.xlabel('Unit ID')
    plt.ylabel('Total Number of Spikes')
    plt.title(f'Ranking of Unit IDs based on Total Number of Spikes ({epoch}, {stim_level})')
    
    # Show the plot
    plt.show()

    # Return the ranked unit IDs and their total number of spikes
    return ranked_units

# Usage
# Replace 'your_epoch_here' and 'your_stim_level_here' with the actual epoch and stim level you want to use
# ranked_units = plot_and_rank_units(analysis_instance, 'your_epoch_here', 'your_stim_level_here')


# Usage
# Replace 'your_epoch_here' and 'your_stim_level_here' with the actual epoch and stim level you want to use
ranked_units = plot_and_rank_units(analysis_instance, 'Pre', 'Max')



In [ ]:
#get the bottom 10 units
bottom_10_units = ranked_units.tail(10)
bottom_10_units

In [ ]:
#now remove the bottom units from the lmc_opsin group
analysis_instance.remove_cell_ids('Lmc_opsin', ['cid238', 'cid79','cid91', 'cid156' ])

In [ ]:
analysis_instance.compare_percent_changes_between_groups('Lmc_opsin', 'Lmc_noopsin')

In [ ]:
result_df
#now index in cid81 and get the 

In [ ]:
#query the data frame to get columns names with 'fold_change' in them
#result_df.filter(like='fold_change_normalized', axis=1)
result_df.filter(like='percent_change_normalized', axis=1)

In [ ]:
# Plot the ranked units based on a specific metric
analysis_instance.plot_ranked_units('Lmc_opsin', 'Post_Mid_fold_change', ascending=True)

In [ ]:
#plot the mean firing rate for the mid stimulation level
analysis_instance.plot_mean_firing('Low', group_name='Lmc_opsin') 



In [ ]:
#plot the standard deviation for the mid stimulation level
analysis_instance.plot_standard_deviation('Pooled', group_name='Lmc_opsin')

In [ ]:
#now iterate for all the stims for the Lmc_opsin group and plot the mean firing rate
for stim in ['Zero', 'Low', 'Mid', 'Max', 'Pooled']:
    analysis_instance.plot_mean_firing(stim, group_name='Lmc_opsin')
    

In [ ]:
#repeat this for the standard deviation
for stim in ['Zero', 'Low', 'Mid', 'Max', 'Pooled']:
    analysis_instance.plot_standard_deviation(stim, group_name='Lmc_opsin')
    

In [ ]:
#now plot the fold change for the Lmc_opsin group
for stim in ['Zero', 'Low', 'Mid', 'Max', 'Pooled']:
    analysis_instance.plot_fold_change(stim, group_name='Lmc_opsin')

In [ ]:
#create a for loop to plot the fold change for all the stimulation levels
for stim in ['Zero', 'Low', 'Mid', 'Max', 'Pooled']:
    analysis_instance.plot_fold_change_plotly(stim, group_name='Lmc_opsin')

In [ ]:
analysis_instance.plot_fold_change_plotly('Max', group_name='Lmc_opsin')


    

In [ ]:
analysis_instance.plot_standard_deviation_plotly('Low', group_name='Lmc_opsin', data_type='both')

In [ ]:
analysis_instance.plot_mean_firing_rate_plotly('Max', group_name='Lmc_opsin')

In [ ]:
analysis_instance.remove_cell_ids('Lmc_opsin', ['cid238', 'cid79'])

In [ ]:
analysis_instance.plot_fold_change_plotly('Pooled', group_name='Lmc_opsin')


In [ ]:
analysis_instance.analysis_df

In [ ]:
analysis_instance.plot_group_responses('Lmc_opsin')

In [ ]:
analysis_instance.plot_group_responses_histogram('Lmc_opsin', nbinsx=100)

In [ ]:
analysis_instance.plot_group_responses_histogramsubplots('Lmc_opsin', nbinsx=20)


In [ ]:
from dash import Dash, dcc, html, Input, Output
import plotly.graph_objects as go
from plotly.subplots import make_subplots

app = Dash(__name__)

# Get the unique unit IDs for the specified group
all_unit_ids = list(unit_ids)

# Define the layout of the app
app.layout = html.Div([
    dcc.Checklist(
        id='unit-selection',
        options=[{'label': str(unit), 'value': str(unit)} for unit in all_unit_ids],
        value=all_unit_ids  # Initially, all checkboxes are selected
    ),
    dcc.Dropdown(
        id='stimulation-selection',
        options=[{'label': stim_level, 'value': stim_level} for stim_level in ['Zero', 'Low', 'Mid', 'Max', 'Pooled']],
        value='Zero'  # Initially, 'Zero' is selected
    ),
    dcc.Graph(
        id='histogram-plot'
    )
])

# Define the callback to update the plot based on the selected units
@app.callback(
    Output('histogram-plot', 'figure'),
    Input('unit-selection', 'value'),
    Input('stimulation-selection', 'value'),
    Input('histogram-plot', 'relayoutData'),
)
def update_plot(selected_units, current_stimulation, relayoutData):
    # Define the stimulation levels and phases
    stim_levels = ['Zero', 'Low', 'Mid', 'Max', 'Pooled']
    phases = ['early', 'late']
    epochs = ['Pre', 'Post']

    # Function to get pooled data for the current selection of units
    def get_pooled_data(epoch, phase, stim_level, visible_unit_ids):
        pooled_data = []
        for unit_id in visible_unit_ids:
            data = analysis_df.loc[(group_name, unit_id), f'{epoch}_{stim_level}_{phase}_response']
            pooled_data.extend(data)
        return pooled_data

    # Create a subplot with 2 rows and 2 columns
    fig = make_subplots(rows=2, cols=2, subplot_titles=('Pre Early', 'Pre Late', 'Post Early', 'Post Late'))

    # Loop through each condition to plot the histograms
    for i, epoch in enumerate(epochs):
        for j, phase in enumerate(phases):
            data = get_pooled_data(epoch, phase, current_stimulation, selected_units)
            fig.add_trace(go.Histogram(x=data, nbinsx=20, name='Pooled Data'), row=i+1, col=j+1)

    # If the user has zoomed in, maintain the same x-axis range
    if relayoutData:
        for i, key in enumerate(relayoutData.keys()):
            if 'xaxis.range' in key:
                fig.update_xaxes(range=relayoutData[key], row=i//2+1, col=i%2+1)

    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
